In [1]:
# default_exp calib

This contains scripts to perform camera calibration

# Import

In [2]:
# export
import numpy as np
import torch

from camera_calib_python.control_refine import CheckerRefiner
from camera_calib_python.modules import (CamSF, Heikkila97Distortion,
                                         Normalize, Rigid)
from camera_calib_python.utils import *

In [3]:
from pathlib import Path

import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace

from camera_calib_python.cb_geom import CbGeom, CpCSRGrid, FmCFPGrid
from camera_calib_python.control_refine import OpenCVCheckerRefiner
from camera_calib_python.fiducial_detect import DotVisionCheckerDLDetector
from camera_calib_python.image import File16bitImg

# Utility

In [4]:
def init_intrin(Hs, sz):
    yo, xo = (np.array(sz)-1)/2
    po_inv = np.array([[1, 0, -xo],
                       [0, 1, -yo],
                       [0, 0,   1]])
    A, b = [np.empty(0) for _ in range(2)]
    for H in Hs:
        H_bar = po_inv@H
        v1, v2 = H_bar[:,0], H_bar[:,1]
        v3, v4 = v1+v2, v1-v2
        v1, v2, v3, v4 = unitize(np.stack([v1, v2, v3, v4]))
        A = np.r_[A, np.array([v1[0]*v2[0]+v1[1]*v2[1], v3[0]*v4[0]+v3[1]*v4[1]])]
        b = np.r_[b, np.array([-v1[2]*v2[2], -v3[2]*v4[2]])]
    alpha = np.sqrt(np.dot(b,A)/np.dot(b,b))
    return np.array([[alpha,     0, xo],
                     [    0, alpha, yo],
                     [    0,     0,  1]])

In [5]:
def init_extrin(H, A):
    H_bar = np.linalg.inv(A)@H
    lambdas = np.linalg.norm(H_bar, axis=0)
    r1, r2 = [H_bar[:,idx]/lambdas[idx] for idx in range(2)]
    r3 = np.cross(r1, r2)
    R = approx_R(np.c_[r1,r2,r3])
    t = H_bar[:,2]/np.mean(lambdas[0:2])
    return R, t

# Single Calibration

This will calibrate a single camera

In [6]:
def single_calib_f(imgs, 
                   cb_geom, 
                   detector, 
                   refiner, 
                   Cam=CamSF,
                   Distortion=lambda:Heikkila97Distortion(*torch.zeros(4, dtype=torch.double)),
                   loss=torch.nn.functional.mse_loss,
                   cutoff_it=500,
                   cutoff_norm=1e-5):
    ps_f_w = cb_geom.ps_f
    
    # Get initial homographies via fiducial markers
    Hs = []
    for img in imgs:
        ps_f_p = detector(img.array_gs)
        Hs.append(homography(ps_f_w, ps_f_p))
        
    return single_calib_H(imgs, 
                          cb_geom, 
                          Hs, 
                          refiner,
                          Cam=CamSF,
                          Distortion=Distortion,
                          loss=loss,
                          cutoff_it=cutoff_it,
                          cutoff_norm=cutoff_norm)

In [7]:
def single_calib_H(imgs, 
                   cb_geom, 
                   Hs,
                   refiner,
                   Cam=CamSF,
                   Distortion=lambda:Heikkila97Distortion(*torch.zeros(4, dtype=torch.double)),
                   loss=torch.nn.functional.mse_loss,
                   cutoff_it=500,
                   cutoff_norm=1e-5):
    ps_c_w = cb_geom.ps_c
    bs_c_w = cb_geom.bs_c
    
    # Get refined control points
    pss_c_p = []
    for img, H in zip(imgs, Hs):
        print(f'Refining control points for: {img.name}...')
        ps_c_p = pmm(H, ps_c_w, aug=True)
        bs_c_p = np.array([pmm(H, b_c_w, aug=True) for b_c_w in bs_c_w], np.object)
        pss_c_p.append(refiner(img.array_gs, ps_c_p, bs_c_p))

    # Update homographies with refined control points
    for idx, ps_c_p in enumerate(pss_c_p):
        Hs[idx] = homography(ps_c_w, ps_c_p)
    
    # Get initial guess for intrinsics; distortion assumed to be zero
    A = init_intrin(Hs, imgs[0].size)
    
    # Get intial guess for extrinsics
    Rs, ts = [], []
    for H in Hs:
        R, t = init_extrin(H, A)
        Rs.append(R)
        ts.append(t)
        
    # Entering torch land...
        
    # Get points for nonlinear refinement    
    ps_c_w = torch.tensor(np.c_[ps_c_w, np.zeros(len(ps_c_w))], dtype=torch.double) # 3rd dimenion is zero for calibration boards
    pss_c_p = [torch.tensor(ps_c_p, dtype=torch.double) for ps_c_p in pss_c_p]
    
    # Intrinsic modules
    cam = Cam(torch.tensor(A, dtype=torch.double))
    distort = Distortion()
    
    # Extrinsic modules
    rigids = [Rigid(torch.tensor(R, dtype=torch.double), torch.tensor(t, dtype=torch.double)) for R,t in zip(Rs,ts)]
    
    # Get ms_w2p transformations
    if isinstance(refiner, CheckerRefiner):
        ms_w2p = [torch.nn.Sequential(rigids[idx], 
                                      Normalize(), 
                                      distort, 
                                      cam)
                  for idx in range(len(rigids))]
    else:
        raise RuntimeError(f'Dont know how to handle: {type(refiner)}')

    pass
        
    # Do nonlinear optimization
    def _get_loss():
        ls, total = [], 0
        for m_w2p, ps_c_p in zip(ms_w2p, pss_c_p):
            idx = torch.all(torch.isfinite(ps_c_p), dim=1)
            ls.append(loss(ps_c_p[idx], m_w2p(ps_c_w[idx]))*len(idx))
            total += len(idx)
        return torch.sum(torch.stack(ls))/total 
    
    def _get_params():
        return sum([list(m.parameters()) for m in [cam, distort]+rigids], [])
    
    set_trace()
        
    print(f'Refining parameters...')
    optim = torch.optim.LBFGS(_get_params())
    params_prev = torch.cat([p.view(-1) for p in _get_params()])
    for it in range(cutoff_it):
        def _closure():
            optim.zero_grad()
            l = _get_loss()
            l.backward()
            return l
        optim.step(_closure)
        params = torch.cat([p.view(-1) for p in _get_params()])
        norm = torch.norm(params-params_prev)
        print(f' - Iteration: {it:03d} - Norm: {norm.item():10.5f} - Loss: {_get_loss().item():10.5f}')
        if norm < cutoff_norm: break
        params_prev = params
        
    return cam, distort, rigids, torch2np((tuple(pss_c_p), tuple([m_w2p(ps_c_w) for m_w2p in ms_w2p])))

# Test

In [8]:
h_cb = 50.8
w_cb = 50.8
h_f = 42.672
w_f = 42.672
num_c_h = 16
num_c_w = 16
spacing_c = 2.032
cb_geom = CbGeom(h_cb, w_cb,
                 CpCSRGrid(num_c_h, num_c_w, spacing_c),
                 FmCFPGrid(h_f, w_f))

In [9]:
file_model = Path('/home/justin/justinblaber/camera_calib_python/models/dot_vision_checker.pth')
detector = DotVisionCheckerDLDetector(file_model)

In [10]:
refiner = OpenCVCheckerRefiner(hw_min=5, hw_max=15, cutoff_it=5, cutoff_norm=1e-2)

In [11]:
#imgs = [File16bitImg(file_img) for file_img in Path('/home/justin/Downloads/CAM_1').glob('*CAM_1*.png')]

In [12]:
imgs = [File16bitImg(file_img) for file_img in Path('data/dot_vision_checker').glob('*CAM_1*.png')]

In [13]:
cam, distort, rigids, debug = single_calib_f(imgs, cb_geom, detector, refiner)

Refining control points for: SERIAL_19061245_DATETIME_2019-06-07-00:38:19-438594_CAM_1_FRAMEID_0_COUNTER_1...
> <ipython-input-7-bb9c701178f0>(74)single_calib_H()
     72     set_trace()
     73 
---> 74     print(f'Refining parameters...')
     75     optim = torch.optim.LBFGS(_get_params())
     76     params_prev = torch.cat([p.view(-1) for p in _get_params()])

ipdb> l = _get_loss()
*** Error in argument: '= _get_loss()'
ipdb> ! l = _get_loss()
*** IndentationError: unexpected indent
ipdb> !l = _get_loss()
ipdb> !l.backward()
ipdb> !cam.x_o.grad
tensor(-0.0028, dtype=torch.float64)
ipdb> !cam.y_o.grad
tensor(0.0044, dtype=torch.float64)
ipdb> !cam.alpha.grad
tensor(-0.0002, dtype=torch.float64)
ipdb> cam.y_o
Parameter containing:
tensor(767.5000, dtype=torch.float64, requires_grad=True)
ipdb> rigids
[Rigid(
  (ms): ModuleList(
    (0): EulerRotation(ex:-0.724 ey:0.1416 ez:2.303)
    (1): Translation(tx:12.01 ty:2.679 tz:183.1)
  )
)]
ipdb> rigids[0]
Rigid(
  (ms): ModuleList(
    (

BdbQuit: 

In [ ]:
for p1, p2 in zip(debug[0], debug[1]):
    res = (p1-p2)
    plt.plot(res[:,0], res[:,1], 'gs')

# Build

In [ ]:
!nbdev_build_lib